NOTES:
- 'checkpoint' file is not important if u load checkpoints directly by name
- Now train with minimun 5 keypoints per person
- Make sure to clean out temp/ before start training a new model

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-813416f0-5aa8-b3df-e73a-70a572b3c233)


GPU 0: Tesla P100-PCIE-16GB 

In [2]:
!unzip ./drive/MyDrive/single-person-pose-estimation/dataset/tfrecords.zip

Archive:  ./drive/MyDrive/single-person-pose-estimation/dataset/tfrecords.zip
   creating: dataset/tfrecords/
   creating: dataset/tfrecords/valid/
  inflating: dataset/tfrecords/valid/file_valid_02-1551.tfrec  
  inflating: dataset/tfrecords/valid/file_valid_01-2048.tfrec  
  inflating: dataset/tfrecords/valid/file_valid_00-2048.tfrec  
   creating: dataset/tfrecords/train/
  inflating: dataset/tfrecords/train/file_train_21-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_00-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_47-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_50-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_25-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_20-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_62-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_52-2048.tfrec  
  inflating: dataset/tfrecords/train/file_train_36-2048.tfrec  
  inflating: dataset/tfrecords/train/file_trai

In [1]:
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
sys.path.append('/content/drive/MyDrive/single-person-pose-estimation')
from configs import default_config as cfg



---



---
Dataset


In [3]:
from dataset_builder import DatasetBuilder

In [4]:
ds_builder = DatasetBuilder(cfg)

Train dataset with 66 tfrecords and 134214 examples.
Valid dataset with 3 tfrecords and 5647 examples.




---



---
Model


In [5]:
from model.hourglass import create_hourglas_model

In [6]:
model = create_hourglas_model(cfg.NUM_KEYPOINTS, 
                                 cfg.HG_NUM_STACKS,
                                 cfg.HG_NUM_CHANNELS,
                                 cfg.IMAGE_SHAPE,
                                 cfg.HM_ACTIVATION)

Created Hourglass model:
    1. 4 stacks.
    2. 12922052 parameters. Call model.get_summary() for more detail.
    




---



---
TRAINER


In [7]:
from trainer import Trainer



---
Initial train (run once at the start)


In [8]:
trainer = Trainer(model, ds_builder, epochs = 15, learning_rate = 1e-3, loss_str = 'mse', config = cfg)

Mean Squared Error


In [ ]:
trainer.train()

First training with:
    1. Current date 30-05-2022.
    2. Number of epochs 15.
    3. Batch size 16.
    4. Optimizer configs: {'name': 'Adam', 'learning_rate': 0.001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
    

Learning rate for epoch 1 is 0.0010000000474974513
Epoch 1/15
8388/8388 [==============================] - ETA: 0s - loss: 0.0108 - hg0_conv_1x1_predict_loss: 0.0030 - hg1_conv_1x1_predict_loss: 0.0028 - hg2_conv_1x1_predict_loss: 0.0026 - hg3_conv_1x1_predict_loss: 0.0025
Epoch 1: val_loss improved from inf to 0.00401, saving model to temp/checkpoints/best_val_loss_weights.ckpt
8388/8388 [==============================] - 3467s 409ms/step - loss: 0.0108 - hg0_conv_1x1_predict_loss: 0.0030 - hg1_conv_1x1_predict_loss: 0.0028 - hg2_conv_1x1_predict_loss: 0.0026 - hg3_conv_1x1_predict_loss: 0.0025 - val_loss: 0.0040 - val_hg0_conv_1x1_predict_loss: 0.0010 - val_hg1_conv_1x1_predict_loss: 9.9071e-04 - val_hg2_conv_1x1_predict_loss: 0.0

In [ ]:
# to drive
!cp -r ./temp ./drive/MyDrive/single-person-pose-estimation



---
Resume train



In [ ]:
# from drive
!cp -r ./drive/MyDrive/single-person-pose-estimation/temp ./

In [ ]:
trainer = Trainer(model, ds_builder, epochs = 15, learning_rate = 0.5e-3, loss_str = 'weighted_mse', config = cfg)

In [ ]:
trainer.resume_training()

In [ ]:
# to drive
!cp -r ./temp ./drive/MyDrive/single-person-pose-estimation




---


---


Test prediction

Should reset runtime


In [ ]:
# from drive
!cp -r ./drive/MyDrive/single-person-pose-estimation/temp ./

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
trainer = Trainer(model, ds_builder, epochs = 20, learning_rate = 0.001, config = cfg)

In [ ]:
#loaded_model = trainer.get_best_weights_model()
loaded_model = trainer.get_lattest_weights_model()

In [ ]:
ds = ds_builder.get_ds_prediction() 

In [ ]:
loaded_model.evaluate(ds, steps = np.ceil(ds_builder.num_valid_examples // cfg.BATCH_SIZE))

In [ ]:
pred1, pred2, pred3, pred4 = loaded_model.predict(ds)

In [ ]:
it = iter(ds)

In [ ]:
images_batch, heatmaps_batch = next(it)

In [ ]:
#Truth ground
plt.figure(figsize=(20,20))
for i in range(16):
    ax = plt.subplot(16/4, 16/4, i + 1)
    plt.imshow(images_batch[i], cmap = 'gray', alpha = 0.5)
    h = tf.image.resize(heatmaps_batch[i], (256, 256))
    plt.imshow(tf.reduce_sum(h, axis = -1), alpha = 0.5)
    plt.axis("off")

In [ ]:
#RME loss

plt.figure(figsize=(20,20))
for i in range(16):
    ax = plt.subplot(16/4, 16/4, i + 1)
    plt.imshow(images_batch[i], cmap = 'gray', alpha = 0.5)
    p = tf.image.resize(pred4[i+16], (256,256))
    plt.imshow(tf.reduce_sum(p, axis = -1), alpha = 0.5)
    plt.axis("off")



---

Save the whole model

In [ ]:
trainer.save_model('trained_models/first_Adam_RME')

In [ ]:
!cp -r ./trained_models ./drive/MyDrive/single-person-pose-estimation 